## Numeric Optimization : A Connection to Deep Learning

...Does this even work? seems like gradient methods fail or produce degenerate solutions for these type of problems...



Up until now, we've avoided using the usual forward and backward pass notation to define our variables, as i wanted to draw a link between other timeseries methods. It turns out the *smoothened* hidden state distributions are the $\gamma_t(\cdot)$ variable used in HMM literature. But let's unroll the helper variables one by one and start with the *forward* variable $\alpha$, which is a **unnormalized likelihood** $\ell(X_t|Y_{0:t})$.

\begin{align*}
    \alpha_t(j) &\overset{\text{Def.}}{=} (\sum_{i=1}^N \alpha_{t-1}(i)\mathcal A[i, j]) \mathcal B[j, Y_t]
\end{align*}

We obtain the full data likelihood $P(Y|\theta)$, by summing over $\alpha_T(i)$. As all the computation is differentiable (only matrix multiplications), we can **back-propagate** through the computation graph and optimize via **gradient descent** using auto-diff and **Optimization constraints**.


<p align="center"><img src="numeric_optim.drawio.png" alt="drawing" width="500"/></p>

In [1]:
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import minimize
from functools import partial

def likelihood(Bs : np.ndarray, y_i : np.ndarray, lengths : np.ndarray) -> np.ndarray:
    """
    Bs : np.ndarray     of shape num_states * prod(num_obs_i).
    offsets : List[int] contains num_obs - 1 indeces for the split.
    returns: product of likelihoods for observation y_i
    
    """
    sections = np.insert(np.cumsum(lengths)[:-1], 0, 0)
    indeces = y_i + sections
    
    arrs = Bs[:, indeces]

    return np.prod(arrs, axis=1).squeeze()

def neg_likelihood(pi, A, Bs, Y : NDArray[np.int64], lengths : NDArray[np.int64]) -> np.float32:
    """
        Y : np.ndarray of shape (T, number of emission signals)
    """
    alpha_tm1 : np.ndarray = pi * likelihood(Bs, Y[0, :], lengths)

    for y_i in Y[1:]:
        alpha_tm1 = A.T @ alpha_tm1 * likelihood(Bs, np.array(y_i), lengths)

    return -np.sum(alpha_tm1)

In [2]:
num_states = 2
lengths = np.array([2, 3])
observations = np.array([[0, 1], [1, 0], [1, 1], [1, 0]]) # (D, not B), (C, B), (C, not B), (W, B)

init_A = np.random.random(size=(num_states, num_states))
init_A /= init_A.sum(axis=1)[:, None]

Bs = [
    np.random.random(size=(num_states, M)) for M in lengths
]

init_Bs = [
    B / B.sum(axis=1)[:, None] for B in Bs
]

init_pi = np.ones(shape=(num_states,)) / num_states

assert np.all([np.all([np.isclose(B.sum(axis=1), 1)]) for B in init_Bs]), 'Bs not stochastic'
assert np.all(np.isclose(init_A.sum(axis=1), 1)), 'A not stochastic'
assert np.isclose(init_pi.sum(), 1), 'pi not stochastic'

In [3]:
# turn initial guesses into np arrays
A = init_A.reshape(-1)
Bs = np.concatenate(init_Bs, axis=1).reshape(-1)
pi = np.array(init_pi).reshape(-1)

params = np.concatenate([pi, A, Bs]).astype(np.float64)

In [4]:
from scipy.special import softmax

In [5]:
sections = np.cumsum(lengths)[:-1]
def func(params):
    pi = softmax(params[:num_states])
    A = softmax(params[num_states: num_states + num_states**2].reshape(num_states, num_states), axis=1)
    Bs = params[num_states + num_states**2:].reshape(num_states, -1)
    tmp = np.split(Bs, indices_or_sections=sections, axis=1)
    Bs = np.concatenate([softmax(t, axis=1) for t in tmp], axis=1)
    
    return neg_likelihood(pi, A, Bs, Y=observations, lengths=lengths)
func(params)
    

-0.0008095320162447279

In [6]:
results = minimize(func, x0=params)

In [7]:
pi = softmax(results.x[:num_states])
A = softmax(results.x[num_states: num_states + num_states**2].reshape(num_states, num_states), axis=1)
Bs = results.x[num_states + num_states**2:].reshape(num_states, -1)
tmp = np.split(Bs, indices_or_sections=sections, axis=1)
Bs = np.concatenate([softmax(t, axis=1) for t in tmp], axis=1)

pi, A, Bs

(array([1.00000000e+00, 1.57443794e-10]),
 array([[1.62162656e-07, 9.99999838e-01],
        [1.48350553e-05, 9.99985165e-01]]),
 array([[9.99998410e-01, 1.58970096e-06, 2.47026841e-07, 9.99999753e-01,
         6.47434816e-13],
        [2.55023442e-14, 1.00000000e+00, 6.66661418e-01, 3.33338582e-01,
         1.34945182e-11]]))

In [8]:
np.split(Bs, indices_or_sections=sections, axis=1)

[array([[9.99998410e-01, 1.58970096e-06],
        [2.55023442e-14, 1.00000000e+00]]),
 array([[2.47026841e-07, 9.99999753e-01, 6.47434816e-13],
        [6.66661418e-01, 3.33338582e-01, 1.34945182e-11]])]